## Envinroment preparation

First line if **magic** enabling matplotlib inline plots 

In [1]:
%matplotlib inline

Then we have round of inports:
* **pandas** is our main data storage module
* **glob** and **os** are used for filename manipulations

In [2]:
import pandas as pd
import glob
import os

## Files index and identification

In [3]:
filenames = [ os.path.splitext(wholeFilename)[0] for wholeFilename in 
             [ os.path.basename(wholePath) for wholePath in glob.glob("../input/2*.xlsx") ] ]

In [4]:
dataFiles = pd.DataFrame({"filename": filenames})
dataFiles["year"], dataFiles["pollutant"], dataFiles["resolution"] = dataFiles["filename"].str.split('_', 2).str

In [5]:
dataFiles.head()

,filename,year,pollutant,resolution
0,2010_Pb(PM10)_24g,2010,Pb(PM10),24g
1,2007_As(PM10)_24g,2007,As(PM10),24g
2,2005_PM2.5_24g,2005,PM2.5,24g
3,2009_PM2.5_1g,2009,PM2.5,1g
4,2011_NH4+(PM2.5)_24g,2011,NH4+(PM2.5),24g


In [6]:
dataFiles["year"].value_counts()

2012    35
2011    35
2014    35
2013    34
2015    29
2010    25
2009    24
2008    24
2007    18
2006    18
2005    17
2004    17
2003    17
2002    14
2001    11
2000     6
Name: year, dtype: int64

In [7]:
dataFiles["pollutant"].value_counts()

NO2            32
SO2            31
PM10           29
C6H6           28
PM2.5          22
O3             16
NOx            16
Cd(PM10)       15
Ni(PM10)       15
BaP(PM10)      15
Pb(PM10)       14
As(PM10)       14
CO             13
IP(PM10)        8
BjF(PM10)       8
BbF(PM10)       8
BkF(PM10)       8
BaA(PM10)       8
DBahA(PM10)     7
formaldehyd     5
Na+(PM2.5)      4
NH4+(PM2.5)     4
NO3-(PM2.5)     4
Ca2+(PM2.5)     4
OC(PM2.5)       4
Mg2+(PM2.5)     4
Cl              4
EC(PM2.5)       4
SO42            4
K+(PM2.5)       4
PM25            2
Hg(TGM)         2
DBah(PM10)      1
depozycja       1
Jony            1
Name: pollutant, dtype: int64

In [8]:
dataFiles["resolution"].value_counts()

24g            236
1g             113
(PM2.5)-24g      4
(PM2.5)_24g      4
w_PM25_24g       1
Name: resolution, dtype: int64

## Fixing data files identification

In [9]:
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)-24g", 'pollutant'] = "Cl_(PM2.5)"
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)-24g", 'resolution'] = "24g"

In [10]:
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)_24g", 'pollutant'] = "SO42_(PM2.5)"
dataFiles.ix[dataFiles["resolution"] == "(PM2.5)_24g", 'resolution'] = "24g"

In [11]:
dataFiles.ix[dataFiles["resolution"] == "w_PM25_24g", 'pollutant'] = "Jony_w_PM25"
dataFiles.ix[dataFiles["resolution"] == "w_PM25_24g", 'resolution'] = "24g"

Now **resolution** column should be correct:

In [12]:
dataFiles["resolution"].value_counts()

24g    245
1g     113
Name: resolution, dtype: int64

Lets also fix worngly labelled pollutant:

In [13]:
dataFiles.ix[dataFiles["pollutant"] == "PM25", 'pollutant'] = "PM2.5"

In [14]:
dataFiles.describe()

,filename,year,pollutant,resolution
count,359,359,359,358
unique,359,16,34,2
top,2012_NO2_24g,2012,NO2,24g
freq,1,35,32,245


There is still one empty cell in **resolution** column. Lets identify it:

In [15]:
dataFiles[dataFiles["resolution"].isnull()]

,filename,year,pollutant,resolution
312,2015_depozycja,2015,depozycja,NaN


After manually examinign **2015_depozycja** file I found that it cointains new type of data, which will be useless in planned analysis. I decided to remove it from working memory. 

In [16]:
dataFiles.drop(dataFiles[dataFiles["filename"] == "2015_depozycja"].index, inplace=True)

In [17]:
dataFiles.describe()

,filename,year,pollutant,resolution
count,358,358,358,358
unique,358,16,33,2
top,2012_NO2_24g,2012,NO2,24g
freq,1,35,32,245


## Looking for worst measuring station for each pollutant in 2015

In [18]:
importantPollutants = ["PM10", "PM2.5", "O3", "NO2", "SO2", "C6H6", "CO"]
pollutants2015 = dataFiles[(dataFiles["year"] == "2015") & (dataFiles["resolution"] == "1g") & 
                           (dataFiles["pollutant"].isin(importantPollutants))]

In [19]:
pollutants2015

,filename,year,pollutant,resolution
14,2015_NO2_1g,2015,NO2,1g
106,2015_CO_1g,2015,CO,1g
138,2015_SO2_1g,2015,SO2,1g
141,2015_O3_1g,2015,O3,1g
207,2015_C6H6_1g,2015,C6H6,1g
310,2015_PM10_1g,2015,PM10,1g
326,2015_PM25_1g,2015,PM2.5,1g


In [20]:
from tqdm import tqdm

In [21]:
from collections import Counter

In [22]:
#worstStation = {}
#for index, dataRow in tqdm(pollutants2015.iterrows(), total=len(pollutants2015.index)):
#    dataFromFile = pd.read_excel("../input/" + dataRow["filename"] + ".xlsx", skiprows=[1,2])
#    dataFromFile = dataFromFile.rename(columns={"Kod stacji":"Godzina"})
#    dataFromFile = dataFromFile.set_index("Godzina")
#    worstStation[dataRow["pollutant"]] = dataFromFile.max().sort_values(ascending = False).index[0]

## Building one big data frame

In [23]:
pollutants = importantPollutants
years = sorted(list(dataFiles["year"].unique()))

In [24]:
pollutantsYears = dataFiles[(dataFiles["year"].isin(years)) & (dataFiles["resolution"] == "1g") & 
                           (dataFiles["pollutant"].isin(pollutants))]

In [25]:
bigDataFrame = pd.DataFrame()
for dataYear in years: 
    print(dataYear)
    yearDataFrame = pd.DataFrame()
    for index, dataRow in tqdm(pollutantsYears[pollutantsYears["year"] == dataYear].iterrows(), total=len(pollutantsYears[pollutantsYears["year"] == dataYear].index)):
        data = pd.read_excel("../input/" + dataRow["filename"] + ".xlsx", skiprows=[1,2])
        data = data.rename(columns={"Kod stacji":"Hour"})

        year = int(dataRow["year"])
        rng = pd.date_range(start = str(year) + '-01-01 01:00:00', end = str(year+1) + '-01-01 00:00:00', freq='H')

        # workaround for 2006_PM2.5_1g, 2012_PM10_1g, 2012_O3_1g
        try:
            data["Hour"] = rng
        except ValueError:
            print("File {} has some mess with timestamps".format(dataRow["filename"]))
            continue

        data = data.set_index("Hour")
        data = data.stack()
        data = pd.DataFrame(data, columns=[dataRow["pollutant"]])
        data.index.set_names(['Hour', 'Station'], inplace=True)

        yearDataFrame = pd.concat([yearDataFrame, data], axis=1)
    
    bigDataFrame = bigDataFrame.append(yearDataFrame)

  0%|          | 0/2 [00:00<?, ?it/s]

2000


  0%|          | 0/3 [00:00<?, ?it/s]

2001


  0%|          | 0/3 [00:00<?, ?it/s]

...

100%|██████████| 7/7 [03:02<00:00, 28.48s/it]


2014


  0%|          | 0/7 [00:00<?, ?it/s]

2015


100%|██████████| 7/7 [03:32<00:00, 28.52s/it]


In [26]:
bigDataFrame.to_pickle("../output/bigDataFrame.pkl")

In [27]:
bigDataFrame

C6H6        CO        NO2  \
Hour                Station                                                
2000-01-01 01:00:00 Pm.a01a                     NaN       NaN  25.000000   
                    Pm.a03a                     NaN       NaN        NaN   
2000-01-01 02:00:00 DsCzer02                    NaN       NaN   6.000000   
                    DsJelw05                    NaN       NaN  14.000000   
                    DsSniezka                   NaN       NaN        NaN   
                    LbJarczew                   NaN       NaN        NaN   
                    MpKrakowWIOSAKra6117        NaN       NaN  62.000000   
                    MpKrakowWIOSBulw6118        NaN       NaN  48.000000   
                    MpKrakowWIOSPrad6115        NaN       NaN  38.000000   
                    MzWarPodIMGW                NaN       NaN        NaN   
                    MzWarszKrucza               NaN       NaN        NaN   
                    Pm.a01a                     NaN       NaN  24.000000   
                    Pm.a02a                     NaN       NaN  27.000000   
                    Pm.a03a                     NaN       NaN  13.000000   
                    Pm.a04a                     NaN       NaN  11.000000   
                    Pm.a06a                     NaN       NaN  13.000000   
                    Pm.a07a                     NaN       NaN   7.000000   
                    Pm.a08a                     NaN       NaN  11.000000   
                    Pm.a09a                     NaN       NaN  10.000000   
                    Pm08LEBAiEMEPa              NaN       NaN        NaN   
                    SlBytomByto_modrz           NaN       NaN  55.000000   
                    SlKatowKato_kossu           NaN       NaN        NaN   
                    SlSosnoSosn_narut           NaN       NaN  55.000000   
                    SlUstroCies_sanat           NaN       NaN        NaN   
                    WmPuszcz_IOS_Borecka        NaN       NaN        NaN   
2000-01-01 03:00:00 DsCzer02                    NaN       NaN   3.000000   
                    DsJelw05                    NaN       NaN  12.000000   
                    DsSniezka                   NaN       NaN        NaN   
                    LbJarczew                   NaN       NaN        NaN   
                    MpKrakowWIOSAKra6117        NaN       NaN  56.000000   
...                                             ...       ...        ...   
2016-01-01 00:00:00 SlKatoKossut                NaN       NaN  48.290000   
                    SlKatoPlebA4                NaN  1.295760  63.356200   
                    SlRybniBorki          10.985600  0.837930  20.880100   
                    SlSosnoLubel                NaN       NaN  51.530000   
                    SlTychyTolst                NaN       NaN  33.202800   
                    SlUstronSana                NaN       NaN   2.934920   
                    SlWodzGalczy                NaN  2.095970  45.876400   
                    SlZabSkloCur                NaN  1.491160  39.449300   
                    SlZlotPotLes                NaN       NaN   8.782320   
                    SlZorySikors                NaN  1.438560        NaN   
                    SlZywieKoper                NaN       NaN  20.399000   
                    WmElbBazynsk           1.679240  0.290620   6.775208   
                    WmGoldJacwie                NaN  0.381811  10.383073   
                    WmMragParkow                NaN       NaN        NaN   
                    WmOlsPuszkin           0.658847  0.462765  11.895022   
                    WmOstrChrobr                NaN  0.554376  10.615822   
                    WmPuszczaBor                NaN       NaN        NaN   
                    WpBoroDrapal                NaN       NaN  18.765600   
                    WpKaliSawick                NaN  0.644600  19.310900   
                    WpKoniWyszyn                NaN  0.550630  14.855800   
                   